# Test plane-based registration
Test different plane-based registration methods

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import torch

import planeslam.io as io
from planeslam.general import plot_3D_setup, color_legend
from planeslam.scan import pc_to_scan
from planeslam.registration import get_correspondences, extract_corresponding_features
from planeslam.geometry.util import quat_to_rot_mat

%load_ext autoreload
%autoreload 2

In [2]:
np.set_printoptions(suppress=True)

### Load AirSim data

In [3]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [4]:
# Extract scans
num_scans = len(PC_data)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

### Decoupled

In [5]:
from planeslam.registration import decoupled_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = decoupled_register(scans[i+1], scans[i])
    avg_runtime += time.time() - start_time
    print(t_hat)
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

[[ 0.85379642]
 [-0.17429398]
 [ 0.46246713]]
[[ 2.95168808]
 [-0.01658027]
 [-1.13556727]]
[[ 4.16863735]
 [ 0.01398406]
 [-0.3381851 ]]
[[ 3.96851642]
 [ 0.02358479]
 [-0.21498566]]
[[-6.45533312]
 [-0.03374595]
 [ 0.13797484]]
[[-16.45433056]
 [ -0.07251653]
 [  0.3252316 ]]
[[ 4.50080014]
 [ 0.04057502]
 [-0.2373203 ]]
[[ 4.86580561]
 [-0.0120256 ]
 [-0.04346649]]
[[ 4.38636989]
 [ 0.03460902]
 [-0.02325109]]
[[ 4.28062767]
 [-0.2014159 ]
 [ 0.00087601]]
[[ 4.51646742]
 [ 0.14195673]
 [-0.08669257]]
[[  5.94084976]
 [  0.00633556]
 [-11.38948788]]
[[17.5669326]
 [-0.1952477]
 [23.4005   ]]
[[1224.54902213]
 [   3.76937261]
 [5354.59938478]]
[[-0.56798381]
 [ 0.06901303]
 [-0.10303344]]
[[ 0.45722702]
 [ 0.07071825]
 [-0.61991248]]
[[0.54583353]
 [0.02485906]
 [0.03897516]]
[[ 0.05576875]
 [-0.02552299]
 [-0.0209569 ]]
[[-0.25355929]
 [ 0.005749  ]
 [ 0.03889906]]
average registration time:  0.0019023167459588301


In [6]:
np.round(abs_traj_transformations[:,:3,3],2)

array([[   0.85,   -0.17,    0.46],
       [   3.72,   -0.19,   -0.54],
       [   7.92,   -0.19,   -0.56],
       [  11.89,   -0.14,   -0.69],
       [   5.44,   -0.19,   -0.47],
       [ -11.01,   -0.27,   -0.14],
       [  -6.51,   -0.25,   -0.38],
       [  -1.65,   -0.25,   -0.37],
       [   2.74,   -0.22,   -0.4 ],
       [   7.02,   -0.41,   -0.39],
       [  11.55,   -0.28,   -0.06],
       [   1.01,   -0.27,   -0.95],
       [  17.11,   -0.46,   22.09],
       [1244.04,    3.38, 5334.57],
       [ 802.58,   13.32, 5418.39],
       [ -15.5 ,    3.96, 5476.88],
       [-247.31,    3.89, 5471.33],
       [  27.86,   -2.1 , 5476.83],
       [ 175.31,    8.84, 5474.12]])

### Gauss-newton

In [7]:
from planeslam.registration import GN_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = GN_register(scans[i+1], scans[i])
    avg_runtime += time.time() - start_time
    print(t_hat)
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

final loss:  50.915949995953035
[[0.20814988]
 [0.81964319]
 [4.53891009]]
final loss:  1460.8760816870224
[[-11.73404653]
 [-25.61669028]
 [-22.10287004]]
final loss:  3140.430017915746
[[-49.50294962]
 [ -6.2333338 ]
 [-13.26212067]]
final loss:  55.525155298503535
[[-0.28823455]
 [-4.68488924]
 [ 0.18193579]]
final loss:  8.878771252248819e+21
[[ 3.50166917e+11]
 [-5.60100265e+08]
 [ 4.72843683e+10]]
final loss:  1.2274803509714462e+27
[[3.27274130e+13]
 [3.69322221e+12]
 [2.10948451e+13]]
final loss:  15.388914471082941
[[ 2.94310535]
 [-0.0789341 ]
 [ 3.75036889]]
final loss:  90.31597323298082
[[-4.2912856 ]
 [-0.25879911]
 [-0.05292925]]
final loss:  79.65392762461742
[[-4.1499494 ]
 [-0.31995602]
 [-0.33474122]]
final loss:  98.30265778715545
[[-3.43310486]
 [ 1.96639634]
 [ 2.43135789]]
final loss:  78.5344663422902
[[-2.95012889]
 [ 2.32639405]
 [ 1.83958083]]
final loss:  25893.167983646646
[[ 95183.35063485]
 [   562.23586984]
 [-72285.58780128]]
final loss:  1.938455679254

### Torch GN

In [8]:
# Set torch device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

In [9]:
from planeslam.registration import torch_GN_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = torch_GN_register(scans[i+1], scans[i], device)
    avg_runtime += time.time() - start_time
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

final loss:  tensor(0.0562, device='cuda:0')
final loss:  tensor(5.0333e-05, device='cuda:0')
final loss:  tensor(0.0012, device='cuda:0')
final loss:  tensor(0.0126, device='cuda:0')
final loss:  tensor(0.0001, device='cuda:0')
final loss:  tensor(0.3438, device='cuda:0')
final loss:  tensor(0.5796, device='cuda:0')
final loss:  tensor(0.0003, device='cuda:0')
final loss:  tensor(0.0008, device='cuda:0')
final loss:  tensor(0.1881, device='cuda:0')
final loss:  tensor(0.0348, device='cuda:0')
final loss:  tensor(4.2065e+11, device='cuda:0')
final loss:  tensor(3.3330e-05, device='cuda:0')
final loss:  tensor(3.5238, device='cuda:0')
final loss:  tensor(0.0540, device='cuda:0')
final loss:  tensor(0.0483, device='cuda:0')
final loss:  tensor(0.0043, device='cuda:0')
final loss:  tensor(0.0011, device='cuda:0')
final loss:  tensor(0.0401, device='cuda:0')
average registration time:  0.24849431138289602


In [13]:
abs_traj_transformations[:,:3,3]

array([[  0.85299265,  -0.17528395,   0.4618974 ],
       [  3.84827679,  -0.19072411,  -0.90316043],
       [  7.23622477,   1.02220526,   1.64474637],
       [ 11.20611008,   1.04529018,   1.5034847 ],
       [ 11.14946046,   1.04443902,   1.40398059],
       [ 11.30748222,   1.02823482,   1.58089609],
       [ 15.80431205,   1.08082907,   1.5471937 ],
       [ 20.663718  ,   1.15532767,   1.51204148],
       [ 25.05322228,   1.12648621,   1.4957988 ],
       [ 29.338946  ,   0.93173002,   1.47504867],
       [ 33.8748454 ,   1.16077866,   1.02516113],
       [ 30.0497896 ,   1.21746099, -14.1831296 ],
       [ 25.05381002,   0.98455002,  21.81754757],
       [-20.04768327,   0.76102463,  26.41463726],
       [-18.3853372 ,   0.75777405,  27.8675198 ],
       [-13.56092905,   0.84553648,  29.68701503],
       [-11.74401225,   0.85843937,  30.27447838],
       [-11.76487857,   0.60795748,  30.22684749],
       [-12.82945643,   0.56151729,  29.9475938 ]])

In [12]:
filepath = os.path.join(os.getcwd(), '..', '..', 'data', 'results', 'l2l_abs_traj_transformations')
np.save(filepath, np.array(abs_traj_transformations))

### Torch SGD

In [11]:
from planeslam.registration import torch_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = torch_register(scans[i+1], scans[i], device)
    avg_runtime += time.time() - start_time
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

Final loss:  0.05606671  iterations:  250
Final loss:  0.0009951675  iterations:  72
Final loss:  4.0509787  iterations:  250
Final loss:  0.0126964105  iterations:  250
Final loss:  0.0010112934  iterations:  250
Final loss:  0.5292629  iterations:  250
Final loss:  0.5791519  iterations:  250
Final loss:  0.00072609185  iterations:  20
Final loss:  0.0009025088  iterations:  94
Final loss:  0.18806247  iterations:  250
Final loss:  0.038541116  iterations:  250
Final loss:  0.002161515  iterations:  250
Final loss:  0.00479942  iterations:  250
Final loss:  0.11253662  iterations:  250
Final loss:  0.053971708  iterations:  250
Final loss:  0.04811879  iterations:  250
Final loss:  0.0040603136  iterations:  250
Final loss:  0.005883817  iterations:  250
Final loss:  0.040063124  iterations:  250
average registration time:  1.039142043966996


In [12]:
abs_traj_transformations[:,:3,3]

array([[  0.85299265,  -0.17528395,   0.4618974 ],
       [  3.84827679,  -0.19072411,  -0.90316043],
       [  7.23622477,   1.02220526,   1.64474637],
       [ 11.20611008,   1.04529018,   1.5034847 ],
       [ 11.14946046,   1.04443902,   1.40398059],
       [ 11.30748222,   1.02823482,   1.58089609],
       [ 15.80431205,   1.08082907,   1.5471937 ],
       [ 20.663718  ,   1.15532767,   1.51204148],
       [ 25.05322228,   1.12648621,   1.4957988 ],
       [ 29.338946  ,   0.93173002,   1.47504867],
       [ 33.8748454 ,   1.16077866,   1.02516113],
       [ 30.0497896 ,   1.21746099, -14.1831296 ],
       [ 25.05381002,   0.98455002,  21.81754757],
       [-20.04768327,   0.76102463,  26.41463726],
       [-18.3853372 ,   0.75777405,  27.8675198 ],
       [-13.56092905,   0.84553648,  29.68701503],
       [-11.74401225,   0.85843937,  30.27447838],
       [-11.76487857,   0.60795748,  30.22684749],
       [-12.82945643,   0.56151729,  29.9475938 ]])

In [14]:
drone_positions

array([[ 0.        ,  0.        , -2.31653333],
       [ 1.15311325,  0.00000001, -2.02929187],
       [ 4.08561325,  0.00000007, -1.91195583],
       [ 8.04186153,  0.00000016, -2.00577402],
       [12.3312254 ,  0.00000006, -2.06440759],
       [16.72789192, -0.00000015, -2.06920481],
       [21.15950203, -0.00000019, -2.04602313],
       [25.60265923, -0.00000008, -2.0111804 ],
       [30.54411697, -0.00000005, -1.9673357 ],
       [34.99310684, -0.00000005, -1.92621136],
       [39.44282913, -0.        , -1.88452327],
       [43.83399582,  0.00000004, -1.84139442],
       [45.85028076,  0.00000002, -2.32305813],
       [45.31489182,  0.        , -1.83186007],
       [43.51328278,  0.00000004, -1.95378637],
       [42.99415588,  0.00000008, -1.90824699],
       [43.43586349,  0.00000005, -1.85206699],
       [43.86270905, -0.00000002, -1.85960984],
       [43.87223434, -0.00000002, -1.84669709],
       [43.69779968,  0.00000004, -1.84473574]])